In [1]:
print("hello")

hello


In [14]:
import os
os.chdir("../")

In [13]:
%pwd

'C:\\Users\\gedda\\OneDrive\\Desktop\\New folder\\Project\\Medical-chatbot-2'

In [17]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
cd "C:\Users\gedda\OneDrive\Desktop\New folder\Project\Medical-chatbot-2"

C:\Users\gedda\OneDrive\Desktop\New folder\Project\Medical-chatbot-2


In [29]:
#pip install langchain-community
#!pip install pypdf
!pip install sentence-transformers


  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached huggingface_hub-0.27.0-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.27.0-py3-none-any.whl (450 kB)
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/203.1 MB 1.9 MB/s eta 0:01:49
   ---------------------------------------- 0.8/203.1 MB 1.1 MB/s eta 0:03:01
   ---------------------------------------- 1.0/203.1 MB 1.1 MB/s eta 0:03:09
   ------------------------------------

In [18]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [23]:
extracted_data=load_pdf_file(data='C:\\Users\\gedda\\OneDrive\\Desktop\\New folder\\Project\\Medical-chatbot-2\\Data\\')

In [24]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [25]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [26]:
from langchain.embeddings import HuggingFaceEmbeddings

In [27]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [30]:
embeddings = download_hugging_face_embeddings()

c:\Users\gedda\.conda\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [47]:
from dotenv import load_dotenv
load_dotenv()

False

In [69]:
PINECONE_API_KEY=os.environ.get('pcsk_28rw4e_2wfM333krrF7tT8ZLiHDBtq9hFS9Ub9iMj5xX5PrQtxpvE2BFdFZZ1aexmbBtPw')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [71]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_28rw4e_2wfM333krrF7tT8ZLiHDBtq9hFS9Ub9iMj5xX5PrQtxpvE2BFdFZZ1aexmbBtPw")

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
)

In [67]:
#!pip install pinecone
#!pip install pinecone-client
#!pip install google
#!pip install google-api-python-client protobuf google-cloud-storage

#!pip list | grep google
#!pip install --upgrade protobuf
!pip install protoc_gen_openapiv2




  Using cached protoc_gen_openapiv2-0.0.1-py3-none-any.whl.metadata (1.5 kB)
Using cached protoc_gen_openapiv2-0.0.1-py3-none-any.whl (7.9 kB)


In [53]:
import pinecone
print(pinecone.__version__)

5.4.2


In [54]:
pip install --upgrade pinecone-client


Note: you may need to restart the kernel to use updated packages.


In [85]:
#!pip install langchain_pinecone
!pip install langchain_openai

   ---------------------------------------- 0.0/884.2 kB ? eta -:--:--
   ----------------------------------- ---- 786.4/884.2 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 884.2/884.2 kB 4.0 MB/s eta 0:00:00


In [87]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [79]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [80]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [81]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [82]:
retrieved_docs = retriever.invoke("What is Acne?")

In [83]:
retrieved_docs

[Document(id='c4321d25-e4f2-4588-aeb7-b3a27b3383ef', metadata={'page': 39.0, 'source': 'C:\\Users\\gedda\\OneDrive\\Desktop\\New folder\\Project\\Medical-chatbot-2\\Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='422b0252-aabd-4d55-b357-51e25a5c8954', metadata={'page': 39.0, 'source': 'C:\\Users\\gedda\\OneDrive\\Desktop\\New folder\\Project\\Medical-chatbot-2\\Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='dd762f0a-0c84-4ab1-8aa2-86a570c44e74', metadata={'page': 38.0, 'source': 'C:\\Users\\gedda\\OneDrive\\Desktop\\New folder\\Project\\Medical-chatbot-2\\Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Bi

In [88]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [89]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [91]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [92]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [93]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [94]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}